In [1]:
# mount Google Drive

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# unzip dataset folder

import zipfile

def unzip_dataset(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

zip_path = '/content/drive/MyDrive/FYP/Datasets/PETA.zip'
extract_path = '/content/drive/MyDrive/FYP/Datasets'

unzip_dataset(zip_path, extract_path)

In [2]:
# import necessary packages

import os
import shutil
import glob

In [ ]:
# create one common dataset folder from the sub-folders in PETA dataset

peta_dataset_path = '/content/drive/MyDrive/FYP/Datasets/PETA dataset'
output_folder_path = '/content/drive/MyDrive/FYP/Datasets/PETA-merged'

# create output folder if it does not exist
os.makedirs(output_folder_path, exist_ok=True)

# find all 'archive' subdirectories in the PETA dataset folder
archive_folders = glob.glob(os.path.join(peta_dataset_path, '**/archive'), recursive=True)

for archive_folder in archive_folders:
    # find all image files in the 'archive' folder (could be .bmp, .jpg, or .png)
    image_files = glob.glob(os.path.join(archive_folder, '*.[jJ][pP][gG]')) \
                  + glob.glob(os.path.join(archive_folder, '*.[pP][nN][gG]')) \
                  + glob.glob(os.path.join(archive_folder, '*.[jJ][pP][eE][gG]')) \
                  + glob.glob(os.path.join(archive_folder, '*.[bB][mM][pP]'))

    # copy image files
    for image_file in image_files:
        # creates a new name for the image file in the format: <archive_folder_name>_<original_file_name>
        new_name = os.path.join(output_folder_path, '{}_{}'.format(os.path.basename(os.path.dirname(archive_folder)), os.path.basename(image_file)))
        shutil.copy2(image_file, new_name)

In [ ]:
# count the no. of image files

folder_path = '/content/drive/MyDrive/FYP/Datasets/PETA-merged'

image_files = glob.glob(os.path.join(folder_path, '*.[jJ][pP][gG]')) \
              + glob.glob(os.path.join(folder_path, '*.[pP][nN][gG]')) \
              + glob.glob(os.path.join(folder_path, '*.[jJ][pP][eE][gG]')) \
              + glob.glob(os.path.join(folder_path, '*.[bB][mM][pP]'))

num_image_files = len(image_files)

print("Number of image files: ", num_image_files)

Number of image files:  19000


In [ ]:
# create a class directory for pedestrians

!mkdir "/content/drive/MyDrive/FYP/Datasets/PETA-merged/pedestrian"

In [ ]:
# copy images to the new directory

!cp "/content/drive/MyDrive/FYP/Datasets/PETA-merged/"* "/content/drive/MyDrive/FYP/Datasets/PETA-merged/pedestrian/"

cp: -r not specified; omitting directory '/content/drive/MyDrive/FYP/Datasets/PETA-merged/pedestrian'


In [ ]:
# count the no. of image files in pedestrian class folder
# the total no. of images should be 20,052

folder_path = '/content/drive/MyDrive/FYP/Datasets/PETA-merged/pedestrian'

image_files = glob.glob(os.path.join(folder_path, '*.[jJ][pP][gG]')) \
              + glob.glob(os.path.join(folder_path, '*.[pP][nN][gG]')) \
              + glob.glob(os.path.join(folder_path, '*.[jJ][pP][eE][gG]')) \
              + glob.glob(os.path.join(folder_path, '*.[bB][mM][pP]'))

num_image_files = len(image_files)

print("Number of image files: ", num_image_files)

Number of image files:  20052


In [ ]:
# copy HazyPerson images to PETA dataset directory

!cp "/content/drive/MyDrive/Dataset/HazyPerson/Images/"* "/content/drive/MyDrive/FYP/Datasets/PETA-merged/pedestrian/"

In [3]:
# install necessary packages

!pip install torchvision
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.7 MB/s eta 0:00:00


In [4]:
# clone DINO repository from my GitHub repository
# made some changes to the DINO code

!git clone https://github.com/RavindishaWi/dino.git
%cd dino

Cloning into 'dino'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 190 (delta 68), reused 62 (delta 59), pack-reused 106
Receiving objects: 100% (190/190), 24.45 MiB | 21.83 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/content/dino


In [ ]:
# create a new folder 'ssl_finetuning' to save the trained weights
# download pretrained DINO ViT S/8 weights

%cd /content/drive/MyDrive/FYP

!mkdir ssl_finetuning
!wget -O ssl_finetuning/full_checkpoint.pth https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain_full_checkpoint.pth

/content/drive/MyDrive/FYP
mkdir: cannot create directory ‘ssl_finetuning’: File exists
--2023-07-02 07:06:53--  https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain_full_checkpoint.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.8.35, 13.35.8.29, 13.35.8.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.8.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 352284107 (336M) [application/zip]
Saving to: ‘ssl_finetuning/full_checkpoint.pth’

ssl_finetuning/full 100%[===================>] 335.96M  65.4MB/s    in 5.0s    

2023-07-02 07:06:58 (66.7 MB/s) - ‘ssl_finetuning/full_checkpoint.pth’ saved [352284107/352284107]



In [ ]:
# train DINO for PETA dataset

%cd /content/dino

!python main_dino.py --arch vit_small --batch_size 32 --patch_size 8 --data_path /content/drive/MyDrive/FYP/Datasets/PETA-merged --output_dir /content/drive/MyDrive/FYP/ssl_finetuning

/content/dino
Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main
Will run the code on one GPU.
| distributed init (rank 0): env://
git:
  sha: fe5b8837a0f6a0d01e129e33326e64f20dc91b16, status: clean, branch: main

arch: vit_small
batch_size_per_gpu: 32
clip_grad: 3.0
data_path: /content/drive/MyDrive/FYP/Datasets/PETA-merged
dist_url: env://
drop_path_rate: 0.1
epochs: 100
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 0
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
norm_last_layer: True
num_workers: 10
optimizer: adamw
out_dim: 65536
output_dir: /content/drive/MyDrive/FYP/ssl_finetuning
patch_size: 8
rank: 0
saveckp_freq: 20
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1
Data loaded: there are 20051 images.
Student and Teacher are built: they 

In [ ]:
%cd /content/drive/MyDrive/FYP

!mkdir finetuning_checkpoint

/content/drive/MyDrive/FYP


In [ ]:
import shutil

# specify file paths
source_path = "/content/drive/MyDrive/FYP/ssl_finetuning/checkpoint.pth"
destination_path = "/content/drive/MyDrive/FYP/finetuning_checkpoint/checkpoint.pth"

# copy the file
shutil.copyfile(source_path, destination_path)

'/content/drive/MyDrive/FYP/finetuning_checkpoint/checkpoint.pth'

In [9]:
%cd /content/dino

!python main_dino.py --arch vit_small --batch_size 32 --patch_size 8 --data_path /content/drive/MyDrive/FYP/Datasets/PETA-merged --output_dir /content/drive/MyDrive/FYP/finetuning_checkpoint

/content/dino
Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main
Will run the code on one GPU.
| distributed init (rank 0): env://
git:
  sha: fe5b8837a0f6a0d01e129e33326e64f20dc91b16, status: has uncommited changes, branch: main

arch: vit_small
batch_size_per_gpu: 32
clip_grad: 3.0
data_path: /content/drive/MyDrive/FYP/Datasets/PETA-merged
dist_url: env://
drop_path_rate: 0.1
epochs: 150
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 0
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
norm_last_layer: True
num_workers: 10
optimizer: adamw
out_dim: 65536
output_dir: /content/drive/MyDrive/FYP/finetuning_checkpoint
patch_size: 8
rank: 0
saveckp_freq: 20
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1
Data loaded: there are 20051 images.
Student and 

In [12]:
%cd /content/dino

!python main_dino.py --arch vit_small --batch_size 32 --patch_size 8 --data_path /content/drive/MyDrive/FYP/Datasets/PETA-merged --output_dir /content/drive/MyDrive/FYP/finetuning_checkpoint

/content/dino
Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main
Will run the code on one GPU.
| distributed init (rank 0): env://
git:
  sha: fe5b8837a0f6a0d01e129e33326e64f20dc91b16, status: has uncommited changes, branch: main

arch: vit_small
batch_size_per_gpu: 32
clip_grad: 3.0
data_path: /content/drive/MyDrive/FYP/Datasets/PETA-merged
dist_url: env://
drop_path_rate: 0.1
epochs: 200
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 0
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
norm_last_layer: True
num_workers: 10
optimizer: adamw
out_dim: 65536
output_dir: /content/drive/MyDrive/FYP/finetuning_checkpoint
patch_size: 8
rank: 0
saveckp_freq: 20
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1
Data loaded: there are 20051 images.
Student and 

In [8]:
!python visualize_attention.py --image_path /content/drive/MyDrive/Models/trained_weights/foggy.jpg --pretrained_weights /content/drive/MyDrive/FYP/finetuning_checkpoint/checkpoint0100.pth

Take key teacher in provided checkpoint dict
Pretrained weights found at /content/drive/MyDrive/FYP/finetuning_checkpoint/checkpoint0100.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
./attn-head0.png saved.
./attn-head1.png saved.
./attn-head2.png saved.
./attn-head3.png saved.
./attn-head4.png saved.
./attn-head5.png saved.
/content/dino/combined_attention.png saved.
Output image saved successfully.


In [8]:
!python visualize_attention.py --image_path /content/drive/MyDrive/Models/trained_weights/foggy-weather.jpg

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
./attn-head0.png saved.
./attn-head1.png saved.
./attn-head2.png saved.
./attn-head3.png saved.
./attn-head4.png saved.
./attn-head5.png saved.
/content/dino/combined_attention.png saved.
Output image saved successfully.
